# Random Forest model for Fatality prediction

In [1]:
import pandas as pd

data =  pd.read_csv('COVID-19_Case_Surveillance_Public_Use_Data.csv')

c:\users\sreeje.sreedhar\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
print (len(data))

data = data.dropna()

20565345


In [3]:
# length after dropping na
print (len(data))

2624165


In [4]:
# considering only death yes and no rows
df = data[(data.death_yn == "Yes") | (data.death_yn == "No")]

In [5]:
print (set(df['death_yn'].tolist()))

{'Yes', 'No'}


In [6]:
# Preprocessing (labelling) the input features
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
input_columns = [4,5,6,7,8,9,11]
for col in input_columns:   
    colname = df.columns[col]
    df[colname]= label_encoder.fit_transform(df[colname])

c:\users\sreeje.sreedhar\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [7]:
# Considering the input and the target features
X = df.iloc[:,[4,5,6,7,8,9,11]].values
y = df.iloc[:,10].values

In [8]:
from collections import Counter
#  imbalance in data 
print(Counter(y))

Counter({'No': 1475466, 'Yes': 45264})


In [11]:
#  Oversampled because of imbalance
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_oversample, Y_oversample = oversample.fit_resample(X,y)

In [12]:
print(Counter(Y_oversample))

Counter({'No': 1475466, 'Yes': 1475466})


In [13]:
#  training and testing dataset split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_oversample, Y_oversample, test_size = 0.4, random_state = 21)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [15]:
# model building....
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [16]:
#  testing with the test set
y_pred = classifier.predict(X_test)

## Metrics to analyse the accuracy of the model

In [17]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
original = y_test
answer = y_pred
print (classification_report(original, answer))
print(confusion_matrix(original,answer))


              precision    recall  f1-score   support

          No       0.95      0.90      0.93    590082
         Yes       0.91      0.95      0.93    590291

    accuracy                           0.93   1180373
   macro avg       0.93      0.93      0.93   1180373
weighted avg       0.93      0.93      0.93   1180373

[[533870  56212]
 [ 29936 560355]]


### Sample testing

In [19]:
import numpy as np
current_status = "Laboratory-confirmed case"
sex = "Female"
age_group = "0 - 9 Years"
race_ethnicity_combined = "Unknown"
hosp_yn = "Missing"
icu_yn = "Missing"
medcond_yn = "Missing"

input_test  = [label_encoder.fit_transform([current_status])[0],label_encoder.fit_transform([sex])[0],
          label_encoder.fit_transform([age_group])[0],label_encoder.fit_transform([race_ethnicity_combined])[0],
          label_encoder.fit_transform([hosp_yn])[0], label_encoder.fit_transform([icu_yn])[0],label_encoder.fit_transform([medcond_yn])[0]]

input_test = np.array(input_test)
y_tester= classifier.predict([input_test])
print (y_tester[0])

No
